<a href="https://colab.research.google.com/github/JF11579/Plotly_RE/blob/main/SameHouse/Stacked_Styles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes


# Libraries

In [1]:
import pandas as pd
from google.colab import drive  # <<<<<<<<<------
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

# Options

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change to the directory containing your CSV
%cd /content/drive/MyDrive/DATA/Real_Estate_Data

Mounted at /content/drive
/content/drive/MyDrive/DATA/Real_Estate_Data


In [4]:
data = pd.read_csv('Combined_data_Jan_10_3.csv')

data.head(3)

<ipython-input-4-3a02a300e0fc>:1: DtypeWarning: Columns (5,17,19,21,22,23,24,26,28,29,32,33,35,36,37,41,43,45,46,47,48,49,51,54,55,56,57,59,60,62,72,73,74,76,85,86,89,90,104,107,109,111,113,115,117,123,124,125,127,135,143,144,151,152,154,156,158,160,161,164,167,174,180,181,184,185,187,188,189,190,191,192,193,194,199,200,202,203,208,214,216,217,220) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Combined_data_Jan_10_3.csv')


,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Contract Date,Year,Air_Conditioning,Acres,Property_Description,Agency,Also_For_Rent,Interior_Features,Apt#,Attic_Description,Auxiliary_Rooms,Buyer_Agency_Compensation,Waterfront_Features,Building,Building_Permit,Block,Listing_Broker_Phone_#,Branch,Modifications_Exclusions,Basement_Description,Room_Level,Business_Age,Business_Type,Utilities_Included,Included,Cash_Down,Contract_Cancel_Effective_Date,Under_Steel,Available_Utilities,Class_Description,Co-Listing_Agent,Co-Listing_Agent2,Broker_Code,Common_Charges,Commission_Oblig_Date,Compensation_Paid_By,Compass_Points,W/R_Conditions,Construction_Description,Lot_Description,County,Contract_Price,Cross_Street,Location_Features,DBA,Exterior_Features,Deductible_%,Income_Desc_1,Income_Desc_2,Income_Desc_3,Income_Desc_4,Income_Desc_5,Income_Desc_6,Income_Desc_7,Income_Desc_8,Detached/Attached,Development,District,Days_On_Market,Driveway,Fees,Finance_Restrictions,Final_Mortgage,Fixtures_&_Equipment,Floor_Load,#_Fireplaces,Frontage_Feet,Feet_From_Corner,Heating_Fuel,Parking,LandLord_Pays,Gross_Profit_%,Building_Features,Heating_Type,Hilp$,Inc_1_$,Inc_2_$,Inc_3_$,Inc_4_$,Inc_5_$,Inc_6_$,Inc_7_$,Inc_8_$,Inventory_Included,Insurance,Distribute_To_Internet_Portals,Inventory,Listing_Date,Lease,Topography,Listing_Agent,Number_of_Docks,Lockbox,Lolp$,Lot,Lot_Square_Footage,Lot_Size,Listing_Price,Last_Status_Change,Lease_Years,Last_Update_Date,Maintenance,Maintenance/Repairs_Expense,Minimum_Plot_Size,Miscellaneous_Expense,Model_Name,Moved_From,Mortgage_Type,Negotiate_Direct,Net_Income,New_Construction,#_Families,#_Floors_In_Building,#_Floors_In_Unit,#_Kitchens,#_Plots,#_Units,#_Heating_Zones,Occupancy,Office_%,Broker_Open_House_Description,Broker_Open_Houses,Lease_Conditions,Original_Price,Other_Fees,Out_of_Area_Town,Owner,Owner_Financing,Parking_Charges,Parking_Facilities,Parking_Spaces,Parking_Spaces_At_Building,Payroll_Expense,%_Difference,Permit,Personal_Property_Exclusion,Pets_Allowed,Status/Showing_Phone,Planning_Board_Approved,Pool_Description,Possible_Subdivision,Price_Range_Listing,Property_Type,Proposed_Title_Date,Purchaser,Renting_Broker_Branch,Renting_Broker_Code,Renting_Broker_Compensation,Rented_Date,Road_Surface,Remarks,Rent_Expense,Renting_Agent,Rent_Income,Rent_Per_Square_Foot,Rental_Type,Relist_Obligation_Date,Listing_Broker,Rooms,Rented_Price,Renting_Broker,Sale_or_Rent,Selling_Broker_Branch,Selling_Broker_Code,Seller_Agency_Compensation,School_District_#,Seats,Section,Section/Area,Security_Deposit,Selling_Agent,Separate_Hotwater_Heater,Show_Instructions,Selling_Broker,Street_Direction,Street_Number,Street_Suffix,Status,Stories,Supercedes_ML#,Taxes,Taxes_W/Star_Exemption,Title_Date,TOM,Terms,TOM_Start_Date,TOM_End_Date,Total_Expenses,Total_Income,Type_Ownership,Type_Sale,Unit_#,Unit_on_Floor_#,Vacancy_%,Development_Status,Village_Taxes,Water_Frontage,Water_Description,Wooded/Cleared,Withdrawn/Release_Date,Exp_Date,Year_Built,Zone,Zoning
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,9/14/2006,2006-09-14,2006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Pre-Processing

In [5]:
data = data.dropna(subset=["Sold_Price"]).query("`Sold_Price` != 0")
data = data[['Address', 'Town','Zip','State','Style',
                    'ML#','Sold_Price','Baths_Full', 'Baths_Half',
                    'Bath','Bedrooms',
                      'Appearance','REO' , 'School_District_Name',
                 'Approx_Int_Square_Footage','Street_Name','Contract_Date']]
data= data.loc[data.Town.isin(['Roosevelt', 'Hempstead', 'Freeport', 'Farmingdale', 'Mineola','Westbury', 'Roslyns', 'Port Washington', 'Locust Valley'])]
# Convert 'Contract_Date' to datetime
data['Contract_Date'] = pd.to_datetime(data['Contract_Date'])

# Extract the year
data['Year'] = data['Contract_Date'].dt.year


# Drop rows where 'Year' is NaN
data = data.dropna(subset=['Year'])

# Now convert 'Year' to integer
data['Year'] = data['Year'].astype(int)
####
median_sold_price_all = data.groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_price_all.rename(columns={'Sold_Price': 'Median_Sold_Price_All'}, inplace=True)
#
median_sold_non_ranch = data[data['Style'] != 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_non_ranch.rename(columns={'Sold_Price': 'Median_Sold_Non_Ranch'}, inplace=True)
#
median_sold_ranch = data[data['Style'] == 'Ranch'].groupby(['Town', 'Year'])['Sold_Price'].median().reset_index()
median_sold_ranch.rename(columns={'Sold_Price': 'Median_Sold_Ranch'}, inplace=True)

#######
# Merge Median_Sold_Price_All
data = data.merge(median_sold_price_all, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Non_Ranch
data = data.merge(median_sold_non_ranch, on=['Town', 'Year'], how='left')

# Merge Median_Sold_Ranch
data = data.merge(median_sold_ranch, on=['Town', 'Year'], how='left')

In [6]:
# prompt: create new column called Month_year from Contract_Date

data['Month_Year'] = data['Contract_Date'].dt.strftime('%B-%Y')


## consolidate House Styles

In [7]:
# Calculate the frequency of each style
style_counts = data['Style'].value_counts(normalize=True) * 100

# Identify styles that are less than 5%
styles_to_replace = style_counts[style_counts < 5].index
#
# Replace styles that are less than 5% with 'Other'
data['Style'] = data['Style'].replace(styles_to_replace, 'Other')

In [8]:
data.head(3)

,Address,Town,Zip,State,Style,ML#,Sold_Price,Baths_Full,Baths_Half,Bath,Bedrooms,Appearance,REO,School_District_Name,Approx_Int_Square_Footage,Street_Name,Contract_Date,Year,Median_Sold_Price_All,Median_Sold_Non_Ranch,Median_Sold_Ranch,Month_Year
0,19 Winans Place,Locust Valley,11560,NY,Cape,1870802,459000.0,2.0,0.0,2.0,4.0,Excellent,NaN,Locust Valley,NaN,Winans,2006-09-14,2006,609500.0,610000.0,520000.0,September-2006
1,59 11th Street,Locust Valley,11560,NY,Colonial,2745482,360000.0,2.0,0.0,2.0,4.0,Good,N,Locust Valley,NaN,11th,2015-11-23,2015,485000.0,515000.0,440000.0,November-2015
2,19 White Spots Lane,Locust Valley,11560,NY,Other,2427641,650000.0,2.0,1.0,2.5,3.0,Mint,N,Locust Valley,2353.0,White Spots,2011-10-08,2011,666000.0,770000.0,361000.0,October-2011


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26087 entries, 0 to 26086
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Address                    26087 non-null  object        
 1   Town                       26087 non-null  object        
 2   Zip                        26087 non-null  int64         
 3   State                      26087 non-null  object        
 4   Style                      26085 non-null  object        
 5   ML#                        26087 non-null  object        
 6   Sold_Price                 26087 non-null  float64       
 7   Baths_Full                 26087 non-null  float64       
 8   Baths_Half                 26086 non-null  float64       
 9   Bath                       26080 non-null  float64       
 10  Bedrooms                   26087 non-null  float64       
 11  Appearance                 20299 non-null  object        
 12  REO 

# Plot

In [38]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px  # Ensure Plotly Express is imported

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Get unique styles
unique_styles = style_median_prices['Style'].unique()

# Assign a unique color to each style
colors = px.colors.qualitative.Plotly  # Or any other color sequence
style_color_map = {style: colors[i % len(colors)] for i, style in enumerate(unique_styles)}

# Initialize an empty figure
fig = go.Figure()

# Track which styles have been added to the legend
styles_added_to_legend = set()

# For each town, add a bar for each style in ascending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in ascending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=True)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        show_in_legend = row['Style'] not in styles_added_to_legend
        styles_added_to_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            marker_color=style_color_map[row['Style']],  # Use the assigned color
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside',
            showlegend=show_in_legend  # Only show in legend if not already added
        ))

# Customize the layout
fig.update_layout(
    barmode='relative',  # Use 'relative' to stack bars
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [37]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px  # Ensure Plotly Express is imported

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Get unique styles
unique_styles = style_median_prices['Style'].unique()

# Assign a unique color to each style
colors = px.colors.qualitative.Plotly  # Or any other color sequence
style_color_map = {style: colors[i % len(colors)] for i, style in enumerate(unique_styles)}

# Initialize an empty figure
fig = go.Figure()

# For each town, add a bar for each style in ascending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in ascending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=True)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            marker_color=style_color_map[row['Style']],  # Use the assigned color
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside',
            showlegend=(row['Style'] not in fig.data)  # Show in legend only if not already present
        ))

# Customize the layout
fig.update_layout(
    barmode='relative',  # Use 'relative' to stack bars
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [36]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Get unique styles
unique_styles = style_median_prices['Style'].unique()

# Assign a unique color to each style
colors = px.colors.qualitative.Plotly  # Or any other color sequence
style_color_map = {style: colors[i % len(colors)] for i, style in enumerate(unique_styles)}

# Initialize an empty figure
fig = go.Figure()

# Track which styles have been added to the legend
styles_in_legend = set()

# For each town, add a bar for each style in descending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in descending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            marker_color=style_color_map[row['Style']],  # Use the assigned color
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside',
            showlegend=show_in_legend
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [35]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Track which styles have been added to the legend
styles_in_legend = set()

# For each town, add a bar for each style in descending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in descending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside',
            showlegend=show_in_legend  # Control legend appearance
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [34]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# For each town, add a bar for each style in descending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in descending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [33]:
# Filter data for Roslyns and sort by median sold price
roslyns_data = style_median_prices[style_median_prices['Town'] == 'Roslyns'].sort_values('Sold_Price', ascending=False)
print(roslyns_data)


       Town        Style  Sold_Price
36  Roslyns     Colonial   1080000.0
37  Roslyns        Other    880000.0
39  Roslyns  Split Level    850000.0
38  Roslyns        Ranch    816500.0
35  Roslyns         Cape    585000.0


In [32]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town, add a bar for each style in ascending order of median sold price
for town in sorted_towns:
    # Sort styles within the town by median sold price in ascending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=True)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [31]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Determine a fixed order for the styles based on their overall median sold prices
overall_style_order = style_median_prices.groupby('Style')['Sold_Price'].median().sort_values(ascending=False).index

# Create a dictionary to hold data for each style
style_data = {style: {'x': [], 'y': [], 'text': []} for style in overall_style_order}

# Populate the data dictionaries
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town]
    town_data_sorted = town_data.sort_values('Sold_Price', ascending=False)

    for _, row in town_data_sorted.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        style_data[row['Style']]['x'].append(town)
        style_data[row['Style']]['y'].append(style_count)
        style_data[row['Style']]['text'].append(f"${row['Sold_Price']}")

# Initialize an empty figure
fig = go.Figure()

# Create the traces from the pre-constructed data
for style, data_dict in style_data.items():
    fig.add_trace(go.Bar(
        name=style,
        x=data_dict['x'],
        y=data_dict['y'],
        text=data_dict['text'],
        hoverinfo='text',
        textposition='inside'
    ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()



In [29]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town, add a bar for each style in the sorted order
for town in sorted_towns:
    # Sort styles within the town by median sold price in descending order
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        median_price = row['Sold_Price']

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${median_price}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [28]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# All unique styles
all_styles = style_median_prices['Style'].unique()

# Create dictionaries to hold data for each style
style_data = {style: {'x': [], 'y': [], 'text': []} for style in all_styles}

# Populate the data dictionaries in descending order of median sold price
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        style_data[row['Style']]['x'].append(town)
        style_data[row['Style']]['y'].append(style_count)
        style_data[row['Style']]['text'].append(f"${row['Sold_Price']}")

# Create the traces using the populated data
for style, data_dict in style_data.items():
    fig.add_trace(go.Bar(
        name=style,
        x=data_dict['x'],
        y=data_dict['y'],
        text=data_dict['text'],
        hoverinfo='text',
        textposition='inside'
    ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [27]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# All unique styles
all_styles = style_median_prices['Style'].unique()

# Create dictionaries to hold data for each style
style_data = {style: {'x': [], 'y': [], 'text': []} for style in all_styles}

# Populate the data dictionaries
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=True)

    for _, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        style_data[row['Style']]['x'].append(town)
        style_data[row['Style']]['y'].append(style_count)
        style_data[row['Style']]['text'].append(f"${row['Sold_Price']}")

# Create the traces using the populated data
for style, data_dict in style_data.items():
    fig.add_trace(go.Bar(
        name=style,
        x=data_dict['x'],
        y=data_dict['y'],
        text=data_dict['text'],
        hoverinfo='text',
        textposition='inside'
    ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()




In [24]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town in the sorted list, add a bar for each style in reverse order
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=True)

    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [23]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town in the sorted list, add a bar for each style in descending order of median sold price
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [22]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town in the sorted list, add a bar for each style in descending order of median sold price
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${row['Sold_Price']}",
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [20]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate overall median sold price for each town
town_median_prices = data.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their overall median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)['Town']

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town in the sorted list, add a bar for each style
for town in sorted_towns:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"${row['Sold_Price']}",  # Just the median sold price
            textposition='inside'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [19]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Sort the styles within each town based on their median sold prices
sorted_styles_per_town = style_median_prices.groupby('Town', group_keys=False).apply(
    lambda x: x.sort_values('Sold_Price', ascending=False)
)

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town, add a bar for each style in the sorted order
for town in sorted_styles_per_town['Town'].unique():
    town_data = sorted_styles_per_town[sorted_styles_per_town['Town'] == town]

    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"{row['Style']} in {town}: Median Sold Price: ${row['Sold_Price']}"
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Median Selling Price by Style Per Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [18]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Calculate the median sold price for each town
town_median_prices = style_median_prices.groupby('Town')['Sold_Price'].median().reset_index()

# Step 3: Sort towns based on their median sold price
sorted_towns = town_median_prices.sort_values('Sold_Price', ascending=True)

# Initialize an empty figure
fig = go.Figure()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town in the sorted list, add a bar for each style
for town in sorted_towns['Town']:
    town_data = style_median_prices[style_median_prices['Town'] == town].sort_values('Sold_Price', ascending=False)

    # Calculate the count and median sold price for each style in this town
    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]
        median_price = row['Sold_Price']

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend,
            hoverinfo='text',
            text=f"Median Sold Price: ${median_price}"
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

# Economist-style customizations
economist_red = '#e3120b'
background_color = 'rgb(195, 215, 227)'
font_family = "Arial, sans-serif"

fig.update_layout(
    title={'text': "Colors Arranged According to Median Selling Price for That Style Per That Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)

# Show the plot
fig.show()


In [17]:
# Fix the legend

import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Sort and maintain a separate order for each town
sorted_styles_per_town = style_median_prices.groupby('Town').apply(
    lambda x: x.sort_values('Sold_Price', ascending=False)
).reset_index(drop=True)

# Initialize an empty figure
fig = go.Figure()

# Unique towns
towns = sorted_styles_per_town['Town'].unique()

# Keep track of styles added to legend
styles_in_legend = set()

# For each town, add a bar for each style in the sorted order
for town in towns:
    town_data = sorted_styles_per_town[sorted_styles_per_town['Town'] == town]

    # Calculate the count for each style in this town
    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        show_in_legend = row['Style'] not in styles_in_legend
        styles_in_legend.add(row['Style'])

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v',
            showlegend=show_in_legend
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

############ ECONOMIST ##########################
# Define Economist-style colors and fonts
economist_red = '#e3120b'  # The Economist's signature red color
background_color = 'rgb(195, 215, 227)'  # The specified background color
font_family = "Arial, sans-serif"  # A commonly used font similar to those in publications

# Customize the plot
fig.update_layout(
    title={'text': "Colors Arranged According to Medain Selling Price for That Style Per That Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)
#########################################################################################
# Show the plot
fig.show()


In [11]:
# colrswithin bars are sorted according to median sale price of that style in that town.

import plotly.graph_objects as go
import pandas as pd

# Assuming 'data' is your DataFrame

# Step 1: Calculate median sold prices for each style within each town
style_median_prices = data.groupby(['Town', 'Style'])['Sold_Price'].median().reset_index()

# Step 2: Sort and maintain a separate order for each town
sorted_styles_per_town = style_median_prices.groupby('Town').apply(
    lambda x: x.sort_values('Sold_Price', ascending=False)
).reset_index(drop=True)

# Initialize an empty figure
fig = go.Figure()

# Unique towns
towns = sorted_styles_per_town['Town'].unique()

# For each town, add a bar for each style in the sorted order
for town in towns:
    town_data = sorted_styles_per_town[sorted_styles_per_town['Town'] == town]

    # Calculate the count for each style in this town
    for index, row in town_data.iterrows():
        style_count = data[(data['Town'] == town) & (data['Style'] == row['Style'])].shape[0]

        fig.add_trace(go.Bar(
            name=row['Style'],
            x=[town],
            y=[style_count],
            orientation='v'
        ))

# Customize the layout
fig.update_layout(
    barmode='stack',
    # Add any additional customization here
)

############ ECONOMIST ##########################
# Define Economist-style colors and fonts
economist_red = '#e3120b'  # The Economist's signature red color
background_color = 'rgb(195, 215, 227)'  # The specified background color
font_family = "Arial, sans-serif"  # A commonly used font similar to those in publications

# Customize the plot
fig.update_layout(
    title={'text': "Colors Arranged According to MedainSelling Price for Thtat Style PEr That Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)
#########################################################################################

# Show the plot
fig.show()


In [12]:
import plotly.express as px
import pandas as pd

# Assuming 'data' is your DataFrame

# Group data by 'Town' and 'Style', and count the occurrences
style_counts = data.groupby(['Town', 'Style']).size().reset_index(name='counts')

# Create a categorical type with the sorted order for plotting (if towns are sorted)
# This part is optional and depends on whether you have sorted towns based on median sold prices.
# If you have, uncomment the following two lines:
# median_prices_sorted = median_prices.sort_values(by='Median_Sold_Price_All')['Town']
# style_counts['Town'] = pd.Categorical(style_counts['Town'], categories=median_prices_sorted, ordered=True)

# Create the stacked bar chart
fig = px.bar(style_counts, x='Town', y='counts', color='Style', text_auto=True)

# Customize the plot
# (Add any customization you require here)

############ ECONOMIST ##########################
# Define Economist-style colors and fonts
economist_red = '#e3120b'  # The Economist's signature red color
background_color = 'rgb(195, 215, 227)'  # The specified background color
font_family = "Arial, sans-serif"  # A commonly used font similar to those in publications

# Customize the plot
fig.update_layout(
    title={'text': "Median Sold Prices by Style in Each Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)
#########################################################################################

# Show the plot
fig.show()


In [13]:
import plotly.express as px
import numpy as np

# Assuming 'data' is your DataFrame
# Add a small random jitter to 'Year'
data['Year_jittered'] = data['Year'] + np.random.uniform(-0.3, 0.3, size=len(data))

# Create scatter plot with jittered 'Year'
fig = px.scatter(data, x='Year_jittered', y='Sold_Price', color='Town')

# Customize the plot
# ... (your customization code)

# Adjust x-axis label
fig.update_xaxes(title_text='Year')

# Show the plot
fig.show()


In [14]:
# Create scatter plot
fig = px.scatter(data, x='Month_Year', y='Sold_Price', color='Town')


##############################  ECONOMIST LAYOUT ###############3
# Define Economist-style colors and fonts
economist_red = '#e3120b'  # The Economist's signature red color
background_color = 'rgb(195, 215, 227)'  # The specified background color
font_family = "Arial, sans-serif"  # A commonly used font similar to those in publications


# Customize the plot
fig.update_layout(
    title={'text': "Sold Prices in Each Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)
#########################################################################

# Show the plot
fig.show()

In [15]:
import plotly.express as px

# Create strip plot
fig = px.strip(data, x='Year', y='Sold_Price', color='Town')

# Customize the plot
# ... (your customization code)

##############################  ECONOMIST LAYOUT ###############3
# Define Economist-style colors and fonts
economist_red = '#e3120b'  # The Economist's signature red color
background_color = 'rgb(195, 215, 227)'  # The specified background color
font_family = "Arial, sans-serif"  # A commonly used font similar to those in publications


# Customize the plot
fig.update_layout(
    title={'text': "Sold Prices in Each Town", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
    title_font=dict(size=18, color=economist_red, family=font_family),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font=dict(color='black', family=font_family),
    legend_title="House Styles"
)
#########################################################################

# Show the plot
fig.show()

In [16]:
import plotly.express as px

# Create box plot
fig = px.box(data, x='Year', y='Sold_Price', color='Town')

# Customize the plot
# ... (your customization code)

# Show the plot
fig.show()
